In [2]:
import os, sys, warnings
import concurrent.futures
from openpyxl import Workbook
from multiprocessing import Pool
from pathlib import Path
from IPython.display import Image, display
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

from selenium.webdriver.common.by import By 
from selenium.webdriver.common.keys import Keys


In [3]:
import pandas as pd

In [4]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

In [5]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install().replace("THIRD_PARTY_NOTICES.", "")))

In [6]:
# Def the website i want to extract the table for
url = 'https://www.coingecko.com/pt'
driver.get(url)

In [7]:
# Picking table from Cripto Coins site and creating a variable for the rows in this table
table = driver.find_element(By.XPATH, '/html/body/div[2]/main/div/div[5]/table')
rows = table.find_elements(By.XPATH, './/tbody//tr')

In [8]:
def maybe_float(s):
    try:
        return round(float(s.replace(".", "").replace(",", ".")), 2)
    except (ValueError, TypeError):
        return s

In [9]:
# Creating the function to iterate in rows and cells of the table and appending to the table_data
def process_cells(cells):
    cell_data = []
    for cell in cells:
        img_tag = cell.find_elements(By.TAG_NAME, 'img')
        data_sort = cell.get_attribute('data-sort')
        if img_tag:
            # Extract the image URL (src attribute)
            img_src = img_tag[0].get_attribute('src')
            cell_data.append(img_src)

        if '$' in cell.text:
            cell_splited = cell.text.split(" ")
            for part_splitted in cell_splited:  
                cell_data.append(maybe_float(part_splitted))
        elif data_sort:
            cell_data.append(round(float(data_sort), 2))
        else:
            cell_data.append(cell.text)
    return cell_data

In [13]:
def process_rows(rows):
    table_data = []
    for row in rows:
        cells = row.find_elements(By.XPATH, './/td')
        cells_data = process_cells(cells)
        table_data.append(cells_data)
    return table_data

In [14]:
table_data = process_rows(rows)

In [30]:
headers = [header.text for header in table.find_elements(By.XPATH, './/thead//th')]
headers = [item for item in headers if item]

headers[0] = "IMG Coin"
insertions = [(0, "ID"), (3, "$"), (8, "30 d"), (9, "$"), (11, "$"), (13, "FDV")
]
for index, value in insertions:
    headers.insert(index, value)

In [56]:
df = pd.DataFrame(table_data)
df_treated_with_link = df.loc[:, (df != '').any(axis=0)] #test drop_na
df_treated_with_link = df_treated_with_link.drop(df.columns[4], axis=1)


In [61]:
df_treated_with_link.style\
    .highlight_max(subset=[2], color='green')\
    .background_gradient(cmap='viridis', subset=[4])\
    .set_properties(**{'border': '1.3px solid black', 'color': 'white'})
df_treated_with_link = df_treated_with_link.reset_index(drop=True)

In [62]:
df_treated_with_link.columns = [headers]
#df_treated_with_link

In [63]:
df_treated = df_treated_with_link.drop(columns=['IMG Coin', 'Últimos 7 dias'])

In [64]:
df_treated_with_link

,ID,IMG Coin,Moeda,$,Preço,1 h,24 h,7 d,30 d,$,Volume em 24 h,$,Capitalização de mercado,FDV,Últimos 7 dias
0,1,https://assets.coingecko.com/coins/images/1/st...,Bitcoin\nBTC,US$,63435.46,0.17,-1.10,5.67,0.04,US$,2.331222e+10,US$,1.253415e+12,1.332110e+12,https://www.coingecko.com/coins/1/sparkline.svg
1,2,https://assets.coingecko.com/coins/images/279/...,Ethereum\nETH,US$,2581.49,0.14,-2.52,11.65,-3.68,US$,1.401607e+10,US$,3.106813e+11,3.106774e+11,https://www.coingecko.com/coins/279/sparkline.svg
2,3,https://assets.coingecko.com/coins/images/325/...,Tether\nUSDT,US$,1.00,-0.04,0.02,-0.13,0.05,US$,4.678407e+10,US$,1.192144e+11,1.192153e+11,https://www.coingecko.com/coins/325/sparkline.svg
3,4,https://assets.coingecko.com/coins/images/825/...,BNB\nBNB,US$,588.91,0.06,-3.06,8.50,6.58,US$,8.609178e+08,US$,8.588753e+10,8.588647e+10,https://www.coingecko.com/coins/825/sparkline.svg
4,5,https://assets.coingecko.com/coins/images/4128...,Solana\nSOL,US$,150.10,-0.35,1.13,16.25,-5.50,US$,2.984289e+09,US$,7.035345e+10,8.784145e+10,https://www.coingecko.com/coins/4128/sparkline...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,https://assets.coingecko.com/coins/images/1233...,MultiversX\nEGLD,US$,29.34,0.28,2.36,13.02,-4.11,US$,2.282131e+07,US$,8.048212e+08,8.047958e+08,https://www.coingecko.com/coins/12335/sparklin...
96,97,https://assets.coingecko.com/coins/images/2643...,Starknet\nSTRK,US$,0.45,1.72,10.26,18.05,17.02,US$,1.806562e+08,US$,8.006644e+08,4.498706e+09,https://www.coingecko.com/coins/26433/sparklin...
97,98,https://assets.coingecko.com/coins/images/1775...,Marinade Staked SOL\nMSOL,US$,182.89,-0.65,0.92,14.64,-4.63,US$,2.039222e+07,US$,7.958557e+08,7.958306e+08,https://www.coingecko.com/coins/17752/sparklin...
98,99,https://assets.coingecko.com/coins/images/9672...,Klaytn\nKLAY,US$,0.14,0.39,0.90,4.87,-18.70,US$,1.588926e+07,US$,7.928844e+08,7.958597e+08,https://www.coingecko.com/coins/9672/sparkline...


In [36]:
df_treated.to_excel("file_test_df_cripto_coin.xlsx", index=False)